In [275]:
%load_ext autoreload
%autoreload 2
from Drivers.scale_driver import *
from Drivers.motor_driver import *
#rom Testing.Dispensing_wrapper import *
from Testing.Robot_wrapper import *
from Drivers.pipette_driver import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [231]:
robot = Robot()

ROBOT_IP: 192.168.1.200, VERSION: v2.3.0, PROTOCOL: V1, DETAIL: 6,6,XI1303,MC1303,v2.3.0, TYPE1300: [1, 1]
change protocol identifier to 3


ControllerError had clean
ControllerError had clean


In [ ]:
robot.restart()

ControllerError, code: 11
servo_error_code, servo_id=1, status=1, code=40


In [273]:
robot.initialize()
robot.GoTo_InitialPoint()

[motion_enable], xArm is not ready to move
[set_state], xArm is ready to move


In [192]:
robot.GoTo_InitialPoint()

In [325]:
coms = {
    'scaleCom': 'COM7',
    'motorsCom': 'COM5',
    'pipetteCom': 'COM6'
}

In [326]:
motors = SerialConnection(coms['motorsCom'], 9600, 10)
homogenizer_motor = Motor(motors, 0, max_position=6000)
homogenizer_motor.check_connection()
homogenizer_motor.set_speed(700)

KeyboardInterrupt: 

In [311]:
homogenizer_motor.move_up(100)

Arduino: MOTOR_FINISHED


In [322]:
homogenizer_motor.move_to_top()

Arduino: WARNING: Target below 0. Moving to position 0 instead.


In [317]:
homogenizer_motor.set_home()

Home position set.


In [49]:
pipette = Pipette(coms['pipetteCom'])
pipette.initialize()

scale = Scale(coms['scaleCom'], 9600, 10)
scale.connect()
scale.tare()

In [267]:
#Positionentest
robot.PickUpVial('Vial1')
robot.VialToScale()
robot.ScaleToLiquidRestPoint()



In [257]:
robot.VialToScale()

In [258]:
robot.ScaleToLiquidRestPoint()

In [202]:
robot.PickUpPipette()

In [80]:
#Pipetten Tip greifen -> muss noch gemacht werden 
robot.arm.set_linear_track_pos(200, wait=True)
robot.GoTo_Point("PipettePoint", 30)#Start point
robot.GoTo_Point("PipetteTip1", 10)
robot.GoTo_Tip("4", 10)
robot.arm.set_position(z=-150, relative=True, speed=10, wait=True)
robot.arm.set_position(x=3.5, y=3.5, relative=True, speed=10, wait=True)
robot.arm.set_position(z=-5, relative=True, speed=3, wait=True)
robot.arm.set_position(z=-5, relative=True, speed=3, wait=True)
robot.arm.set_position(z=-10, relative=True, speed=3, wait=True)
robot.arm.set_position(z=-5, relative=True, speed=6, wait=True)
robot.arm.set_position(z=120, relative=True, speed=50, wait=True)


In [203]:
robot.PuttingBackPipette()

In [204]:
robot.LiquidsToMixingPoint()

In [205]:
robot.TurnOnHomogenizer(10)

In [206]:
robot.TurnOffHomogenizer()

In [207]:
robot.MixingPointToLiquids()

In [208]:
robot.CleaningLiquidsToMixer("Isopropanol")

In [209]:
robot.CleaningLiquidsToStorage("Isopropanol")

In [268]:
#Workflow: Mixen
robot.LiquidsToMixingPoint()
homogenizer_motor.move_to_top()
robot.TurnOnHomogenizer(10)
time.sleep(10)
robot.TurnOffHomogenizer()
homogenizer_motor.move_to_bottom()
#Achtung: Hier muss eine Pause geschaffen werden!!
robot.MixingPointToLiquids()

Arduino: MOTOR_FINISHED
Arduino: MOTOR_FINISHED
[SDK][ERROR][2025-03-31 16:47:14][base.py:374] - - wait_feedback, xarm is stop, state=4
[SDK][ERROR][2025-03-31 16:47:14][base.py:374] - - API -> set_relative_position -> code=1, pos=[0, 0, -45.0, 0, 0, 0], radius=-1, velo=20.0, acc=2000
[SDK][ERROR][2025-03-31 16:47:14][base.py:374] - - wait_move, xarm is stop, state=4
[SDK][ERROR][2025-03-31 16:47:14][base.py:374] - - API -> set_modbus_gripper_position(pos=260) -> code=1
[SDK][ERROR][2025-03-31 16:47:24][base.py:374] - - API -> set_relative_position -> code=1, pos=[0, 0, 45.0, 0, 0, 0], radius=-1, velo=20.0, acc=2000
[SDK][ERROR][2025-03-31 16:47:24][base.py:374] - - API -> set_relative_position -> code=1, pos=[0, 95.0, 0, 0, 0, 0], radius=-1, velo=20.0, acc=2000
[SDK][ERROR][2025-03-31 16:47:24][base.py:374] - - API -> set_position -> code=1, pos=[-367.0, -282.0, 250.0, 1.5707963267948966, 1.5707963267948966, 0.0], radius=-1, velo=40.0, acc=2000
[SDK][ERROR][2025-03-31 16:47:24][base.p

In [288]:
homogenizer_motor.move_to_bottom()
print("Ende")

Arduino: MOTOR_FINISHED
Ende


In [ ]:
#Workflow: Cleaning after Mixing
robot.CleaningLiquidsToMixer("Water")
homogenizer_motor.move_to_top()
robot.TurnOnHomogenizer(10)
time.sleep(10)
robot.TurnOffHomogenizer()
homogenizer_motor.move_to_bottom()
robot.CleaningLiquidsToStorage("Water")
robot.CleaningLiquidsToMixer("Isopropanol")
homogenizer_motor.move_to_top()
robot.TurnOnHomogenizer(10)
time.sleep(10)
robot.TurnOffHomogenizer()
homogenizer_motor.move_to_bottom()
robot.CleaningLiquidsToStorage("Isopropanol")


In [ ]:
#Pipetten Workflow
robot.PickUpPipette()
robot.PickUpPipetteTip("1")
robot.MoveToBinder()
pipette.aspirate(1000)
time.sleep(2)
robot.BinderToVialRestPoint()
pipette.dispense(1000)
time.sleep(2)
pipette.blowout()
time.sleep(2)
robot.VialRestPointToBinder()
pipette.aspirate(1000)
time.sleep(2)
robot.BinderToVialRestPoint()
pipette.dispense(1000)
time.sleep(2)
pipette.blowout()
time.sleep(2)
robot.PuttingBackPipetteTip("1")
pipette.eject()
time.sleep(2)
robot.PuttingBackPipette()